# **Lecture 4: Data Cleaning**
---

### **Description**
In this notebook, we will explore how to find and handle null values.

<br>

### **Structure**
**Part 1**: [Finding Nulls](#p1)

**Part 2**: [Removing Nulls](#p2)

**Part 3**: [Imputing Nulls](#p3)



<br>

### **Learning Objectives**
By the end of this notebook, we will:
* Be able to code to find nulls using a variety of Pandas EDA techniques.
* Be able to code to remove and replace null values.



<br>

**Run the code below before continuing.**


In [ ]:
import pandas as pd
import numpy as np
import random

# Set seed for reproducibility
random.seed(123)
np.random.seed(123)

# Function to introduce random NaNs
def introduce_nan(series, nan_fraction=0.1):
    n_nan = int(len(series) * nan_fraction)
    nan_indices = random.sample(range(len(series)), n_nan)
    series.iloc[nan_indices] = np.nan
    return series

# Create a larger dataset with some messy data
n_rows = 100
data = {
    'student_id': np.arange(1, n_rows + 1),  # Unique IDs
    'name': [random.choice(['Angelique', 'Brianna', 'Cristina', 'Daisy', 'Eve', 'Frank']) for _ in range(n_rows)],  # Random names
    'age': [random.choice([12, 14, 15, 16, 17, 18, np.nan]) for _ in range(n_rows)],  # Random ages with NaNs and empty strings
    'grade': [random.choice(['A', 'B', 'C', 'D', 'F', 'Z', '', 'incomplete', np.nan]) for _ in range(n_rows)],  # Grades with NaNs and 'incomplete'
    'absences': [random.choice([0, 1, 2, 3, np.nan]) for _ in range(n_rows)],  # Absences with NaNs and empty strings
    'detention_hours': [random.choice([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 100,  np.nan, 'none']) for _ in range(n_rows)],  # Detention hours with NaNs and 'none'
}

# Create DataFrame
messy_df = pd.DataFrame(data)

# Introduce random NaNs in some of the columns
messy_df['name'] = introduce_nan(messy_df['name'], nan_fraction=0.1)
messy_df['grade'] = introduce_nan(messy_df['grade'], nan_fraction=0.2)
messy_df['absences'] = introduce_nan(messy_df['absences'], nan_fraction=0.1)
messy_df['detention_hours'] = introduce_nan(messy_df['detention_hours'], nan_fraction=0.15)

# Output the messy dataset
messy_df.head(10) # Display first 10 rows for review

<ipython-input-1-1f41fd5db3cd>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series.iloc[nan_indices] = np.nan
<ipython-input-1-1f41fd5db3cd>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series.iloc[nan_indices] = np.nan
<ipython-input-1-1f41fd5db3cd>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series.iloc[nan_indices] = np.nan
<ipython-input-1-1f41fd5db3cd>:13: SettingWithCopyWarning: 
A value is trying to be set on a c

,student_id,name,age,grade,absences,detention_hours
0,1,Angelique,17.0,C,NaN,NaN
1,2,Cristina,17.0,NaN,3.0,1
2,3,Angelique,18.0,,2.0,6
3,4,Daisy,NaN,A,0.0,0
4,5,Cristina,12.0,C,3.0,NaN
5,6,Angelique,18.0,NaN,NaN,6
6,7,Angelique,15.0,A,2.0,10
7,8,NaN,16.0,NaN,1.0,5
8,9,Eve,NaN,incomplete,NaN,8
9,10,Eve,18.0,incomplete,2.0,0


<a name="p1"></a>

---
##**Part 1: Finding Missing Values**
---

#### **Problem #1.1**

Using `.head()`, what do you initially notice about the data?

In [ ]:
# COMPLETE THIS CODE

#### **Problem #1.2**

Using `.info()`, explore how many nulls each column has.

What do you notice?

In [ ]:
# COMPLETE THIS CODE

#### **Problem #1.3**
Which column(s) do you think we could filter using conditionals?

#### **Problem #1.3**
**Together**, let's filter some columns by what normal values we think it should have.

**NOTE**: Most of our columns are not numeric, but we want to apply numeric logic to them. Use `pd.to_numeric()` to convert your columns for the filter.
* Example: `(pd.to_numeric(messy_df['age'], errors='coerce') < 13)`

In [ ]:
# COMPLETE THIS CODE

#### **Problem #1.3**
**Together**, let's filter the data using `isna()`.

In [ ]:
# COMPLETE THIS CODE

<a name="p2"></a>

---
##**Part 2: Dropping Nulls**
---

#### **Problem #2.1**

**Together**, let's drop all nulls from the dataframe row-wise using `dropna()`.

**Note**: `dropna()` does not change our original dataframe, but it instead makes a copy. We either have to save the new dataframe in another variable or set the parameter `inplace = True` which changes our original dataframe. We are going to create a new variable because we want to use the original dataset later on and don't want to permanently modify it.

In [ ]:
# COMPLETE THIS CODE

#### **Problem #2.2**
What do you notice about our dataset now that we've dropped all rows that contained nulls?

<a name="p3"></a>

---
##**Part 3: Imputing Nulls**
---

#### **Problem #3.1**
**Together**, let's impute the missing `age` values with the mean.

In [ ]:
# COMPLETE THIS CODE

#### **Problem #3.2**
**Together**, let's fill the missing `absences` values with the median.

In [ ]:
# COMPLETE THIS CODE

#### **Problem #3.3**
Not all imputations need to be the mean or median, and it can depend on the data what we use to fill the missing values. **Together**, let's impute the missing `detention_hours` values with 0 because we might assume a student has no detention hours if they're missing.

In [ ]:
# COMPLETE THIS CODE

#### **Problem #3.4**
**Together**, let's fill the missing `grade` values with the value `Incomplete`.

In [ ]:
# COMPLETE THIS CODE

#End of notebook
---
© 2024 The Coding School, All rights reserved